# Tools

This notebook explores two powerful features for building capable AI agents with Large Language Models (LLMs): **Tools (Function Calling)** and **Structured Outputs**. We will use the `google-genai` library to interact with Google's Gemini models.

**Learning Objectives:**

1.  **Understand and implement tool use (function calling)** to allow an LLM to interact with external systems.
2.  **Enforce structured data formats (JSON)** from an LLM for reliable data extraction.
3.  **Leverage Pydantic models** to define and manage complex data structures for both function arguments and structured outputs, improving code robustness and clarity.

## 1. Setup

First, let's install the necessary Python libraries.

!pip install -q google-generativeai pydantic python-dotenv

### Configure Gemini API Key

To use the Gemini API, you need an API key. 

1.  Get your key from [Google AI Studio](https://aistudio.google.com/app/apikey).
2.  Create a file named `.env` in the root of this project.
3.  Add the following line to the `.env` file, replacing `your_api_key_here` with your actual key:
    ```
    GEMINI_API_KEY="your_api_key_here"
    ```
The code below will load this key from the `.env` file.

In [3]:
import os
from pathlib import Path

from dotenv import load_dotenv

REPOSITORY_ROOT_DIR = Path().absolute().parent.parent
DOTENV_FILE_PATH = REPOSITORY_ROOT_DIR / ".env"
print(f"Trying to load environment variables from `{DOTENV_FILE_PATH}`")

if not DOTENV_FILE_PATH.exists():
    raise FileNotFoundError(f"Environment file `{DOTENV_FILE_PATH}` not found.")

load_dotenv(dotenv_path=DOTENV_FILE_PATH)

assert "GOOGLE_API_KEY" in os.environ, "`GOOGLE_API_KEY` is not set"

print("Environment variables loaded successfully.")

Trying to load environment variables from `/Users/pauliusztin/Documents/01_projects/TAI/course-ai-agents/.env`
Environment variables loaded successfully.


In [4]:
import json

from google import genai
from google.genai import types
from pydantic import BaseModel, Field

### Initialize the Gemini Client

In [5]:
client = genai.Client()

### Define Constants

We will use the `gemini-2.5-flash` model, which is fast, cost-effective, and supports advanced features like tool use.

In [6]:
MODEL_ID = "gemini-2.5-flash"

DOCUMENT = """
# Q3 2023 Financial Performance Analysis

The Q3 earnings report shows a 20% increase in revenue and a 15% growth in user engagement, 
beating market expectations. These impressive results reflect our successful product strategy 
and strong market positioning.

Our core business segments demonstrated remarkable resilience, with digital services leading 
the growth at 25% year-over-year. The expansion into new markets has proven particularly 
successful, contributing to 30% of the total revenue increase.

Customer acquisition costs decreased by 10% while retention rates improved to 92%, 
marking our best performance to date. These metrics, combined with our healthy cash flow 
position, provide a strong foundation for continued growth into Q4 and beyond.
"""

### Define Pydantic Models

In [7]:
class DocumentMetadata(BaseModel):
    """A class to hold structured metadata for a document."""

    summary: str = Field(description="A concise, 1-2 sentence summary of the document.")
    tags: list[str] = Field(
        description="A list of 3-5 high-level tags relevant to the document."
    )
    keywords: list[str] = Field(
        description="A list of specific keywords or concepts mentioned."
    )

## 2. Implementing tool calls from scratch

LLMs are trained on text and can't perform actions in the real world on their own. **Tools** (or **Function Calling**) are the mechanism we use to bridge this gap. We provide the LLM with a list of available tools, and it can decide which one to use and with what arguments to fulfill a user's request.

The process of calling a tool looks as follows:
1.  **You**: Send the LLM a prompt and a list of available tools.
2.  **LLM**: Responds with a `function_call` request, specifying the tool and arguments.
3.  **You**: Execute the requested function in your code.
4.  **You**: Send the function's output back to the LLM.
5.  **LLM**: Uses the tool's output to generate a final, user-facing response.

### Define Mock Tools

Let's create three simple, mocked functions. One simulates searching Google Drive, one other simulates sending a Discord message and the last one simulates summarizing a document. 

The function signature (input parameters and output type) and docstrings are crucial, as the LLM uses them to understand what each tool does.

In [8]:
def search_google_drive(query: str) -> dict:
    """
    Searches for a file on Google Drive and returns its content or a summary.

    Args:
        query (str): The search query to find the file, e.g., 'Q3 earnings report'.

    Returns:
        str: A JSON string representing the search results, including file names and summaries.
    """

    # In a real scenario, this would interact with the Google Drive API.
    # Here, we mock the response for demonstration.
    return {
        "files": [
            {
                "name": "Q3_Earnings_Report_2024.pdf",
                "id": "file12345",
                "content": DOCUMENT,
            }
        ]
    }


def send_discord_message(channel_id: str, message: str) -> dict:
    """
    Sends a message to a specific Discord channel.

    Args:
        channel_id (str): The ID of the channel to send the message to, e.g., '#finance'.
        message (str): The content of the message to send.

    Returns:
        str: A JSON string confirming the action, e.g., '{"status": "success"}'.
    """

    # Mocking a successful API call
    return {
        "status": "success",
        "channel": channel_id,
        "message_preview": f"{message[:50]}...",
    }


def summarize_financial_report(text: str) -> str:
    """
    Summarizes a financial report.

    Args:
        text (str): The text to summarize.

    Returns:
        str: The summary of the text.
    """

    return "The Q3 earnings report shows an increase in company metrics."

Now we have to define the metadata of each function which will be used as input to the LLM to understand what tool to use and how to call it:

In [9]:
search_google_drive_schema = {
    "name": "search_google_drive",
    "description": "Searches for a file on Google Drive and returns its content or a summary.",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "The search query to find the file, e.g., 'Q3 earnings report'.",
            }
        },
        "required": ["query"],
    },
}

send_discord_message_schema = {
    "name": "send_discord_message",
    "description": "Sends a message to a specific Discord channel.",
    "parameters": {
        "type": "object",
        "properties": {
            "channel_id": {
                "type": "string",
                "description": "The ID of the channel to send the message to, e.g., '#finance'.",
            },
            "message": {
                "type": "string",
                "description": "The content of the message to send.",
            },
        },
        "required": ["channel_id", "message"],
    },
}

summarize_financial_report_schema = {
    "name": "summarize_financial_report",
    "description": "Summarizes a financial report.",
    "parameters": {
        "type": "object",
        "properties": {
            "text": {
                "type": "string",
                "description": "The text to summarize.",
            },
        },
        "required": ["text"],
    },
}

TOOLS = {
    "search_google_drive": {
        "handler": search_google_drive,
        "declaration": search_google_drive_schema,
    },
    "send_discord_message": {
        "handler": send_discord_message,
        "declaration": send_discord_message_schema,
    },
    "summarize_financial_report": {
        "handler": summarize_financial_report,
        "declaration": summarize_financial_report_schema,
    },
}
TOOLS_BY_NAME = {tool_name: tool["handler"] for tool_name, tool in TOOLS.items()}
TOOLS_SCHEMA = [tool["declaration"] for tool in TOOLS.values()]

In [10]:
TOOLS_BY_NAME

{'search_google_drive': <function __main__.search_google_drive(query: str) -> dict>,
 'send_discord_message': <function __main__.send_discord_message(channel_id: str, message: str) -> dict>,
 'summarize_financial_report': <function __main__.summarize_financial_report(text: str) -> str>}

In [11]:
TOOLS_SCHEMA

[{'name': 'search_google_drive',
  'description': 'Searches for a file on Google Drive and returns its content or a summary.',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string',
     'description': "The search query to find the file, e.g., 'Q3 earnings report'."}},
   'required': ['query']}},
 {'name': 'send_discord_message',
  'description': 'Sends a message to a specific Discord channel.',
  'parameters': {'type': 'object',
   'properties': {'channel_id': {'type': 'string',
     'description': "The ID of the channel to send the message to, e.g., '#finance'."},
    'message': {'type': 'string',
     'description': 'The content of the message to send.'}},
   'required': ['channel_id', 'message']}},
 {'name': 'summarize_financial_report',
  'description': 'Summarizes a financial report.',
  'parameters': {'type': 'object',
   'properties': {'text': {'type': 'string',
     'description': 'The text to summarize.'}},
   'required': ['text']}}]

For a better analogy with what we see in frameworks such as LangGraph, let's define a `@tool` decorator that automatically computed the schemas defined above.

In [12]:
from inspect import Parameter, signature
from typing import Any, Callable, Dict, Optional


class ToolFunction:
    def __init__(self, func: Callable, schema: Dict[str, Any]) -> None:
        self.func = func
        self.schema = schema
        self.__name__ = func.__name__
        self.__doc__ = func.__doc__

    def __call__(self, *args: Any, **kwargs: Any) -> Any:
        return self.func(*args, **kwargs)


def tool(description: Optional[str] = None) -> Callable[[Callable], ToolFunction]:
    """
    A decorator that creates a tool schema from a function.

    Args:
        description: Optional override for the function's docstring

    Returns:
        A decorator function that wraps the original function and adds a schema
    """

    def decorator(func: Callable) -> ToolFunction:
        # Get function signature
        sig = signature(func)

        # Create parameters schema
        properties = {}
        required = []

        for param_name, param in sig.parameters.items():
            # Skip self for methods
            if param_name == "self":
                continue

            param_schema = {
                "type": "string",  # Default to string, can be enhanced with type hints
                "description": f"The {param_name} parameter",  # Default description
            }

            # Add to required if parameter has no default value
            if param.default == Parameter.empty:
                required.append(param_name)

            properties[param_name] = param_schema

        # Create the tool schema
        schema = {
            "name": func.__name__,
            "description": description
            or func.__doc__
            or f"Executes the {func.__name__} function.",
            "parameters": {
                "type": "object",
                "properties": properties,
                "required": required,
            },
        }

        return ToolFunction(func, schema)

    return decorator


@tool()
def search_google_drive_example(query: str) -> dict:
    """Search for files in Google Drive."""
    return {"files": []}


@tool()
def send_discord_message_example(channel_id: str, message: str) -> dict:
    """Send a message to a Discord channel."""
    return {"message": "Message sent successfully"}


@tool()
def summarize_financial_report_example(text: str) -> str:
    """Summarize the contents of a financial report."""
    return "Financial report summarized successfully"


tools = [
    search_google_drive_example,
    send_discord_message_example,
    summarize_financial_report_example,
]
for tool_ in tools:
    print(json.dumps(tool_.schema, indent=2))
    print("-" * 50)

{
  "name": "search_google_drive_example",
  "description": "Search for files in Google Drive.",
  "parameters": {
    "type": "object",
    "properties": {
      "query": {
        "type": "string",
        "description": "The query parameter"
      }
    },
    "required": [
      "query"
    ]
  }
}
--------------------------------------------------
{
  "name": "send_discord_message_example",
  "description": "Send a message to a Discord channel.",
  "parameters": {
    "type": "object",
    "properties": {
      "channel_id": {
        "type": "string",
        "description": "The channel_id parameter"
      },
      "message": {
        "type": "string",
        "description": "The message parameter"
      }
    },
    "required": [
      "channel_id",
      "message"
    ]
  }
}
--------------------------------------------------
{
  "name": "summarize_financial_report_example",
  "description": "Summarize the contents of a financial report.",
  "parameters": {
    "type": "object

In [13]:
TOOL_CALLING_SYSTEM_PROMPT = """
You are a helpful AI assistant with access to tools that enable you to take actions and retrieve information to better assist users.

## Tool Usage Guidelines

**When to use tools:**
- When you need information that is not in your training data
- When you need to perform actions in external systems  
- When you need real-time, dynamic, or user-specific data
- When computational operations are required

**Tool selection:**
- Choose the most appropriate tool based on the user's specific request
- If multiple tools could work, select the one that most directly addresses the need
- Consider the order of operations for multi-step tasks

**Parameter requirements:**
- Provide all required parameters with accurate values
- Use the parameter descriptions to understand expected formats and constraints
- Ensure data types match the tool's requirements (strings, numbers, booleans, arrays)

## Tool Call Format

When you need to use a tool, output ONLY the tool call in this exact format:

```tool_call
{{"name": "tool_name", "args": {{"param1": "value1", "param2": "value2"}}}}
```

**Critical formatting rules:**
- Use double quotes for all JSON strings
- Ensure the JSON is valid and properly escaped
- Include ALL required parameters
- Use correct data types as specified in the tool definition
- Do not include any additional text or explanation in the tool call

## Response Behavior

- If no tools are needed, respond directly to the user with helpful information
- If tools are needed, make the tool call first, then provide context about what you're doing
- After receiving tool results, provide a clear, user-friendly explanation of the outcome
- If a tool call fails, explain the issue and suggest alternatives when possible

## Available Tools

<tool_definitions>
{tools}
</tool_definitions>

Remember: Your goal is to be maximally helpful to the user. Use tools when they add value, but don't use them unnecessarily. Always prioritize accuracy and user experience.
"""


Let's try the prompt with a few examples.

In [14]:
USER_PROMPT = """
Can you help me find the latest quarterly report and share key insights with the team?
"""

messages = [TOOL_CALLING_SYSTEM_PROMPT.format(tools=str(TOOLS_SCHEMA)), USER_PROMPT]

response = client.models.generate_content(
    model=MODEL_ID,
    contents=messages,
)

print(response.text)

```tool_call
{"name": "search_google_drive", "args": {"query": "latest quarterly report"}}
```


In [15]:
USER_PROMPT = """
Please find the Q3 earnings report on Google Drive and send a summary of it to 
the #finance channel on Discord.
"""

messages = [TOOL_CALLING_SYSTEM_PROMPT.format(tools=str(TOOLS_SCHEMA)), USER_PROMPT]

response = client.models.generate_content(
    model=MODEL_ID,
    contents=messages,
)
response.text

'```tool_call\n{"name": "search_google_drive", "args": {"query": "Q3 earnings report"}}\n```'

The next step, is to parse the LLM response and call the tool using Python.

First, we parse the LLM output to extract the JSON out of the response:

In [16]:
def extract_tool_call(response_text: str) -> str:
    """
    Extracts the tool call from the response text.
    """
    return response_text.split("```tool_call")[1].split("```")[0].strip()


tool_call_str = extract_tool_call(response.text)
tool_call_str

'{"name": "search_google_drive", "args": {"query": "Q3 earnings report"}}'

Next, we parse the stringified JSON to a python dict:

In [17]:
tool_call = json.loads(tool_call_str)
tool_call

{'name': 'search_google_drive', 'args': {'query': 'Q3 earnings report'}}

Now, we retrieve the tool handler, which is a Python function:

In [18]:
tool_handler = TOOLS_BY_NAME[tool_call["name"]]
tool_handler

<function __main__.search_google_drive(query: str) -> dict>

Ultimately, we call the Python function using the arguments generated by the LLM:

In [19]:
tool_result = tool_handler(**tool_call["args"])
print(json.dumps(tool_result, indent=2))

{
  "files": [
    {
      "name": "Q3_Earnings_Report_2024.pdf",
      "id": "file12345",
      "content": "\n# Q3 2023 Financial Performance Analysis\n\nThe Q3 earnings report shows a 20% increase in revenue and a 15% growth in user engagement, \nbeating market expectations. These impressive results reflect our successful product strategy \nand strong market positioning.\n\nOur core business segments demonstrated remarkable resilience, with digital services leading \nthe growth at 25% year-over-year. The expansion into new markets has proven particularly \nsuccessful, contributing to 30% of the total revenue increase.\n\nCustomer acquisition costs decreased by 10% while retention rates improved to 92%, \nmarking our best performance to date. These metrics, combined with our healthy cash flow \nposition, provide a strong foundation for continued growth into Q4 and beyond.\n"
    }
  ]
}


That's tool calling in a nutshell!

## 3. Implementing tool calls with Gemini

In production, most of the times, we don't implement tool calling from scratch, but leverage the interface of a specific API such as Gemini or OpenAI. Thus, let's see how we can adapt the example from above to Gemini.

In [20]:
def print_user_prompt(user_prompt) -> None:
    print("=" * 75)
    print(f"User Prompt: {user_prompt}".strip())
    print("=" * 75)


def print_function_call(response) -> None:
    response_message_part = response.candidates[0].content.parts[0]

    if hasattr(response_message_part, "function_call"):
        function_call = response_message_part.function_call
        tool_name = function_call.name
        tool_args = function_call.args

        print("=" * 75)
        print(f"Function Call: {function_call}")
        print(f"Function Name: {tool_name}")
        print(f"Function Arguments: {tool_args}")
        print("=" * 75)
    else:
        print("No function call found in the response.")

In [21]:
tools = [
    types.Tool(
        function_declarations=[
            types.FunctionDeclaration(**search_google_drive_schema),
            types.FunctionDeclaration(**send_discord_message_schema),
        ]
    )
]
config = types.GenerateContentConfig(
    tools=tools,
    tool_config=types.ToolConfig(
        function_calling_config=types.FunctionCallingConfig(mode="ANY")
    ),
)

print_user_prompt(USER_PROMPT)
response = client.models.generate_content(
    model=MODEL_ID,
    contents=USER_PROMPT,
    config=config,
)
print_function_call(response)

User Prompt: 
Please find the Q3 earnings report on Google Drive and send a summary of it to 
the #finance channel on Discord.
Function Call: id=None args={'query': 'Q3 earnings report'} name='search_google_drive'
Function Name: search_google_drive
Function Arguments: {'query': 'Q3 earnings report'}


In [22]:
response_message_part = response.candidates[0].content.parts[0]
response_message_part.function_call

FunctionCall(id=None, args={'query': 'Q3 earnings report'}, name='search_google_drive')

In [23]:
tool_handler = TOOLS_BY_NAME[response_message_part.function_call.name]
tool_handler

<function __main__.search_google_drive(query: str) -> dict>

In [24]:
tool_handler(**response_message_part.function_call.args)

{'files': [{'name': 'Q3_Earnings_Report_2024.pdf',
   'id': 'file12345',
   'content': '\n# Q3 2023 Financial Performance Analysis\n\nThe Q3 earnings report shows a 20% increase in revenue and a 15% growth in user engagement, \nbeating market expectations. These impressive results reflect our successful product strategy \nand strong market positioning.\n\nOur core business segments demonstrated remarkable resilience, with digital services leading \nthe growth at 25% year-over-year. The expansion into new markets has proven particularly \nsuccessful, contributing to 30% of the total revenue increase.\n\nCustomer acquisition costs decreased by 10% while retention rates improved to 92%, \nmarking our best performance to date. These metrics, combined with our healthy cash flow \nposition, provide a strong foundation for continued growth into Q4 and beyond.\n'}]}

Now let's aggregate everything into a single function:

In [30]:
def call_tool(function_call) -> dict:
    tool_name = function_call.name
    tool_args = function_call.args

    tool_handler = TOOLS_BY_NAME[tool_name]

    return tool_handler(**tool_args)


def print_tool_result(tool_result) -> None:
    print("=" * 75)
    print(f"Tool Result: {tool_result}")
    print("=" * 75)

In [26]:
print_tool_result(call_tool(response_message_part.function_call))

Tool Result: {'files': [{'name': 'Q3_Earnings_Report_2024.pdf', 'id': 'file12345', 'content': '\n# Q3 2023 Financial Performance Analysis\n\nThe Q3 earnings report shows a 20% increase in revenue and a 15% growth in user engagement, \nbeating market expectations. These impressive results reflect our successful product strategy \nand strong market positioning.\n\nOur core business segments demonstrated remarkable resilience, with digital services leading \nthe growth at 25% year-over-year. The expansion into new markets has proven particularly \nsuccessful, contributing to 30% of the total revenue increase.\n\nCustomer acquisition costs decreased by 10% while retention rates improved to 92%, \nmarking our best performance to date. These metrics, combined with our healthy cash flow \nposition, provide a strong foundation for continued growth into Q4 and beyond.\n'}]}


## 4. Using a Pydantic Model as a Tool for Structured Outputs

A more elegant and powerful pattern is to treat our Pydantic model *as a tool*. We can ask the model to "call" this Pydantic tool, and the arguments it generates will be our structured data.

This combines the power of function calling with the robustness of Pydantic for structured data extraction. It's the recommended approach for complex data extraction tasks.

In [27]:
# The Pydantic class 'DocumentMetadata' is now our 'tool'
extraction_tool = types.Tool(
    function_declarations=[
        types.FunctionDeclaration(
            name="extract_metadata",
            description="Extracts structured metadata from a document.",
            parameters=DocumentMetadata.model_json_schema(),
        )
    ]
)
config = types.GenerateContentConfig(
    tools=[extraction_tool],
    tool_config=types.ToolConfig(
        function_calling_config=types.FunctionCallingConfig(mode="ANY")
    ),
)

prompt = f"""
Please analyze the following document and extract its metadata.

Document:
--- 
{DOCUMENT}
--- 
"""

response = client.models.generate_content(
    model=MODEL_ID, contents=prompt, config=config
)
response_message_part = response.candidates[0].content.parts[0]

if hasattr(response_message_part, "function_call"):
    function_call = response_message_part.function_call
    print("===== Function Call =====")
    print(function_call)

    try:
        document_metadata = DocumentMetadata(**function_call.args)
        print("\n==== Pydantic Validated Object ====")
        print(document_metadata)
    except Exception as e:
        print(f"\nValidation failed: {e}")
else:
    print("The model did not call the extraction tool.")

===== Function Call =====
id=None args={'keywords': ['Q3 2023', 'Revenue increase', 'User engagement', 'Market expectations', 'Product strategy', 'Digital services', 'New markets', 'Customer acquisition costs', 'Retention rates', 'Cash flow'], 'summary': 'The Q3 2023 earnings report highlights a 20% revenue increase and 15% user engagement growth, surpassing market expectations. This success is attributed to a robust product strategy, strong market positioning, and effective expansion into new markets, alongside improved customer acquisition and retention.', 'tags': ['Financials', 'Business Growth', 'Earnings Report', 'Market Performance', 'Company Strategy']} name='extract_metadata'

==== Pydantic Validated Object ====
summary='The Q3 2023 earnings report highlights a 20% revenue increase and 15% user engagement growth, surpassing market expectations. This success is attributed to a robust product strategy, strong market positioning, and effective expansion into new markets, alongside

## 5. Implementing tool calls with Gemini: Running tools in a loop

Now, let's see what happens if we put tool calling in a loop. Let's create a scenario where we ask the agent to perform a multi-step task: find a report on Google Drive and then communicate its findings on Discord.

In [34]:
tools = [
    types.Tool(
        function_declarations=[
            types.FunctionDeclaration(**search_google_drive_schema),
            types.FunctionDeclaration(**send_discord_message_schema),
            types.FunctionDeclaration(**summarize_financial_report_schema),
        ]
    )
]
config = types.GenerateContentConfig(
    tools=tools,
    tool_config=types.ToolConfig(
        function_calling_config=types.FunctionCallingConfig(mode="ANY")
    ),
)


USER_PROMPT = """
Please find the Q3 earnings report on Google Drive and send a summary of it to 
the #finance channel on Discord.
"""

messages = [USER_PROMPT]

print_user_prompt(USER_PROMPT)
response = client.models.generate_content(
    model=MODEL_ID,
    contents=messages,
    config=config,
)
response_message_part = response.candidates[0].content.parts[0]
print_function_call(response)

messages.append(response.candidates[0].content)

# Loop until a function call is still available or we reach the max number of iterations
max_iterations = 3
while hasattr(response_message_part, "function_call") and max_iterations > 0:
    tool_result = call_tool(response_message_part.function_call)
    print_tool_result(tool_result)

    # Add the tool result to the messages creating the following structure:
    # - user prompt
    # - tool call
    # - tool result
    # - tool call
    # - tool result
    # ...
    function_response_part = types.Part(
        function_response=types.FunctionResponse(
            name=response_message_part.function_call.name,
            response=tool_result if isinstance(tool_result, dict) else {"result": tool_result},
        )
    )
    messages.append(function_response_part)

    response = client.models.generate_content(
        model=MODEL_ID,
        contents=messages,
        config=config,
    )

    response_message_part = response.candidates[0].content.parts[0]
    messages.append(response.candidates[0].content)

    print_function_call(response)

    max_iterations -= 1

# 4. Print the final, user-facing answer
print("\n==== Final Agent Response ====")
print(response.candidates[0].content)


User Prompt: 
Please find the Q3 earnings report on Google Drive and send a summary of it to 
the #finance channel on Discord.
Function Call: id=None args={'query': 'Q3 earnings report'} name='search_google_drive'
Function Name: search_google_drive
Function Arguments: {'query': 'Q3 earnings report'}
Tool Result: {'files': [{'name': 'Q3_Earnings_Report_2024.pdf', 'id': 'file12345', 'content': '\n# Q3 2023 Financial Performance Analysis\n\nThe Q3 earnings report shows a 20% increase in revenue and a 15% growth in user engagement, \nbeating market expectations. These impressive results reflect our successful product strategy \nand strong market positioning.\n\nOur core business segments demonstrated remarkable resilience, with digital services leading \nthe growth at 25% year-over-year. The expansion into new markets has proven particularly \nsuccessful, contributing to 30% of the total revenue increase.\n\nCustomer acquisition costs decreased by 10% while retention rates improved to 92%,

Running tools in a loop is amazing, but it assumes the agent needs to run a tool at each step. Also, as the tools might pile up, it doesn't has the chance to think about each tools output, interpret it's result and take a further decision. Here is where ReAct kicks in, which we will explore further in the next lesson.